In [3]:
import math
import numpy as np

def extract_snps_hdf5(h5, ids_ref, markers, diploid=False):
        """Extract genotypes from h5 on ids and markers.
        If diploid, concatenate haplotypes along 0 axis.
        Extract indivuals first, and then subset to SNPs.
        Return 2D array [# haplotypes, # markers]"""
        # Important: Swap of Dimensions [loci<->individuals]
        nblocks = math.ceil(len(markers)/8)
        nsample = len(ids_ref)
        ploidy = 2 if diploid else 1
        gts = np.zeros((ploidy*nsample, nblocks), dtype=np.uint8)
        haplotype_id_with_missing_data = set() # maintain a list of haplotype ids with missing data at any site of interest
        for i in range(nblocks):
            j = min((i+1)*8, len(markers))
            raw_gt = h5["calldata/GT"][markers[i*8:j], :, :ploidy] # can only indexing one dimension at a time, so need to split this into two lines of code
            raw_gt = raw_gt[:, ids_ref, :].reshape((-1, ploidy*nsample)).T
            haplotype_id_with_missing_data.update(np.where(raw_gt == -1)[0])
            gts[:, i] = np.packbits(raw_gt, axis=1).flatten()

        # get rid of haplotypes with missing data
        # eg. male samples only have one chrX
        gts = gts[np.setdiff1d(np.arange(ploidy*nsample), list(haplotype_id_with_missing_data)), :]
        return gts, markers%8

In [3]:
import h5py
import numpy as np

path2h5 = '/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_auto/maf5_chr1.hdf5'
f = h5py.File(path2h5, 'r')
nloci, nsample, ploidy = f['calldata/GT'].shape
print(f['calldata/GT'].shape)
ids_ref = np.array([5,10])
markers = np.sort(np.random.choice(nloci, 11, replace=False))

gts_compact = extract_snps_hdf5(f, ids_ref, markers, diploid=True)
print(f['calldata/GT'].shape)
gts_raw = f['calldata/GT'][markers, :, :]
gts_raw = gts_raw[:, ids_ref, :].reshape((-1, 4)).T
print(gts_compact)
print(gts_raw)
print(np.packbits(gts_raw, axis=1))

(530434, 2504, 2)
(530434, 2504, 2)
(array([[ 56,   0],
       [176,   0],
       [ 24,   0],
       [220,   0]], dtype=uint8), array([5, 3, 7, 2, 5, 7, 7, 2, 5, 4, 0]))
[[0 0 1 1 1 0 0 0 0 0 0]
 [1 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 1 1 0 0 0 0 0 0]
 [1 1 0 1 1 1 0 0 0 0 0]]
[[ 56   0]
 [176   0]
 [ 24   0]
 [220   0]]


In [8]:
def get_ith_bit(number, i):
    # Shift the number i positions to the right and perform a bitwise AND with 1
    # This will isolate the i-th bit
    return (number >> i) & 1

# Example usage:
binary_number = 0b101011   # Example binary number
i = 3                       # Index of the bit to extract (0-based indexing)
result = get_ith_bit(binary_number, i)
print(f"The {i}-th bit of {bin(binary_number)} is: {result}")


The 3-th bit of 0b101011 is: 1


# check my bit-wise operation for allele frequency calculation

In [31]:
import h5py
import numpy as np

path2h5 = '/mnt/archgen/users/yilei/Data/1000G/1000g1240khdf5/all1240/maf5_auto/maf5_chr22.hdf5'
f = h5py.File(path2h5, 'r')
nloci, nsample, ploidy = f['calldata/GT'].shape
print(f['calldata/GT'].shape)
ids_ref = np.arange(nsample)
markers = np.arange(nloci)

blocksize = 8
gts_compact, overhang = extract_snps_hdf5(f, ids_ref, markers, diploid=True)


(97216, 2504, 2)


In [17]:
def allele_freq_per_block(gts_one_block, blocksize):
    """Calculate allele frequency per block.
    Return 1D array of length blocksize"""
    freq = np.zeros(blocksize)
    for i in range(blocksize):
        freq[i] = np.mean(gts_one_block >> (blocksize -1 - i) & 1)
    return freq

allele_freq_per_block(gts_compact[:, -1], blocksize)

array([0.21345847, 0.09704473, 0.10023962, 0.19968051, 0.24001597,
       0.20507188, 0.06908946, 0.12679712])

In [29]:
import time

allelefreq = np.zeros(nloci)
print(gts_compact.shape)
t1 = time.time()
for i in range(nloci):
    index = i//blocksize
    offset = i%blocksize
    allelefreq[i] = np.sum(gts_compact[:, index] >> (blocksize - 1 - offset) & 1)/gts_compact.shape[0]
print(f'for loop takes {time.time()-t1} seconds')

gts = f['calldata/GT'][:, :, :]
gts = gts.reshape((nloci, nsample*2)).T
t1 = time.time()
allelefreq_normal_calculation = np.mean(gts, axis=0)
print(f'normal calculation takes {time.time()-t1} seconds')

# blockwise computation
t1 = time.time()
allelefreq_blockwise = np.concatenate(np.apply_along_axis(allele_freq_per_block, 0, gts_compact, blocksize).T, axis=0)
print(f'blockwise calculation takes {time.time()-t1} seconds')

print(allelefreq)
print(allelefreq_normal_calculation)
print(allelefreq_blockwise)

(5008, 12152)
for loop takes 1.5874099731445312 seconds
normal calculation takes 0.30373620986938477 seconds
blockwise calculation takes 1.7471349239349365 seconds
[0.11242013 0.14117412 0.09384984 ... 0.20507188 0.06908946 0.12679712]
[0.11242013 0.14117412 0.09384984 ... 0.20507188 0.06908946 0.12679712]
[0.11242013 0.14117412 0.09384984 ... 0.20507188 0.06908946 0.12679712]


In [18]:
print(np.isclose(allelefreq - allelefreq_normal_calculation, 0, atol=1e-6).all())

True


In [41]:
import sys
print(sys.getsizeof(gts_compact))
print(sys.getsizeof(gts))
gts_bool = gts.astype(bool)
print(sys.getsizeof(gts_bool))

60857344
128
486857856


In [40]:
ploidy=2
raw_gt = f["calldata/GT"][np.arange(nloci), :, :ploidy]
raw_gt = raw_gt[:, np.arange(nsample), :].reshape((-1, ploidy*nsample)).T
print(sys.getsizeof(raw_gt))
print(raw_gt)

128
[[0 0 0 ... 0 0 0]
 [0 1 0 ... 1 0 0]
 [1 0 1 ... 1 0 0]
 ...
 [1 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
